In [13]:
import pandas as pd
import numpy as np

### Get normalized numeric data

In [50]:
allTrainingDataRaw = pd.read_csv('data/train.csv')

allTrainingLabels = allTrainingDataRaw['Transported']

allTrainingData = allTrainingDataRaw.fillna(value={
    'HomePlanet':'None',
    'CryoSleep':'None',
    'Cabin':'0/0/0',
    'Destination':'None',
    'Age':0,
    'VIP':'False',
    'RoomService':0,
    'FoodCourt':0,
    'ShoppingMall':0,
    'Spa':0,
    'VRDeck':0,
    'Name':'None'
})

allTrainingData[['CabinDeck','CabinNumber','CabinSide']] = allTrainingData['Cabin'].str.split('/').tolist()
allTrainingData['CabinNumber'] = allTrainingData['CabinNumber'].astype('float64')
allTrainingData = allTrainingData.drop(columns=['Cabin','Name'])
allTrainingDataNumeric = allTrainingData.select_dtypes(include='float64').join(
    pd.get_dummies(allTrainingData.select_dtypes(include='object').drop(columns=['PassengerId'])))

allTrainingDataNormalized = allTrainingDataNumeric.apply(lambda x: x/x.max(), axis=0)

valSplitIndex = 8000

trainingData = allTrainingDataNormalized.iloc[:valSplitIndex]
validationData = allTrainingDataNormalized.iloc[valSplitIndex:]

trainingLabels = allTrainingLabels.iloc[:valSplitIndex]
validationLabels = allTrainingLabels.iloc[valSplitIndex:]

### Build model

In [99]:
from keras import models
from keras import layers

inputLen = trainingData.shape[1]

model = models.Sequential()
model.add(layers.Dense(100, activation='relu', input_shape=(inputLen,)))
model.add(layers.Dense(100, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['acc'])

### Train model

In [101]:
history = model.fit(trainingData, trainingLabels,
    epochs=20,
    batch_size=64,
    validation_data=(validationData, validationLabels))

Epoch 1/20
125/125 [==============================] - 0s 2ms/step - loss: 0.3879 - acc: 0.8105
Epoch 2/20
125/125 [==============================] - 0s 2ms/step - loss: 0.3872 - acc: 0.8099
Epoch 3/20
125/125 [==============================] - 0s 2ms/step - loss: 0.3834 - acc: 0.8127
Epoch 4/20
125/125 [==============================] - 0s 2ms/step - loss: 0.3838 - acc: 0.8092
Epoch 5/20
125/125 [==============================] - 0s 2ms/step - loss: 0.3823 - acc: 0.8104
Epoch 6/20
125/125 [==============================] - 0s 2ms/step - loss: 0.3838 - acc: 0.8091
Epoch 7/20
125/125 [==============================] - 0s 2ms/step - loss: 0.3814 - acc: 0.8129
Epoch 8/20
125/125 [==============================] - 0s 2ms/step - loss: 0.3797 - acc: 0.8123
Epoch 9/20
125/125 [==============================] - 0s 2ms/step - loss: 0.3786 - acc: 0.8165
Epoch 10/20
125/125 [==============================] - 0s 2ms/step - loss: 0.3775 - acc: 0.8142
Epoch 11/20
125/125 [============================

### Prepare test data

In [97]:
testDataRaw = pd.read_csv('data/test.csv')

testIds = testDataRaw['PassengerId']

testData = testDataRaw.fillna(value={
    'HomePlanet':'None',
    'CryoSleep':'None',
    'Cabin':'0/0/0',
    'Destination':'None',
    'Age':0,
    'VIP':'False',
    'RoomService':0,
    'FoodCourt':0,
    'ShoppingMall':0,
    'Spa':0,
    'VRDeck':0,
    'Name':'None'
})

testData[['CabinDeck','CabinNumber','CabinSide']] = testData['Cabin'].str.split('/').tolist()
testData['CabinNumber'] = testData['CabinNumber'].astype('float64')
testData = testData.drop(columns=['Cabin','Name'])
testDataNumeric = testData.select_dtypes(include='float64').join(
    pd.get_dummies(testData.select_dtypes(include='object').drop(columns=['PassengerId'])))

testData = testDataNumeric.apply(lambda x: x/x.max(), axis=0)


### Make predictions

In [98]:
import csv

predictions = model.predict(testData).tolist()
results = [[testIds[i], bool(round(predictions[i][0]))] for i in range(len(predictions))]

with open('submissions/mySubmission1.csv','w',newline='') as submissionFile:

    writer = csv.writer(submissionFile, delimiter=',')
    writer.writerow(["PassengerId", "Transported"])

    for result in results:
        writer.writerow(result)

134/134 [==============================] - 0s 761us/step
